In [55]:
import requests

from bs4 import BeautifulSoup

In [57]:
response = requests.get(
    'https://www.sofascore.com/football/match/adana-demirspor-fenerbahce/clbsbmb#id:12528186',
    headers={'User-Agent': 'Mozilla/5.0'} # you'll be blocked if you don't use some type of user agent
)

In [59]:
soup = BeautifulSoup(response.text, 'html.parser')


In [61]:
soup.select('g[cursor="pointer"]')


[]

In [65]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'dnt': '1',
    'if-none-match': 'W/"4bebed6144"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

# you'll need to change the url to match the one you copied. As you can see they have some sort of match id to identify the game --> 10385636
response = requests.get('https://api.sofascore.com/api/v1/event/12528186/shotmap', headers=headers)

In [67]:
response

<Response [200]>

In [69]:
shots = response.json()


In [29]:
shots

{'shotmap': [{'player': {'name': 'Youssef En-Nesyri',
    'firstName': '',
    'lastName': '',
    'slug': 'youssef-en-nesyri',
    'shortName': 'Y. En-Nesyri',
    'position': 'F',
    'jerseyNumber': '19',
    'userCount': 24984,
    'id': 846470,
    'fieldTranslations': {'nameTranslation': {'ar': 'النصيري, يوسف'},
     'shortNameTranslation': {'ar': 'ي. النصيري'}}},
   'isHome': False,
   'shotType': 'save',
   'situation': 'assisted',
   'playerCoordinates': {'x': 6.600105837357415,
    'y': 40.15561547949811,
    'z': 0},
   'bodyPart': 'left-foot',
   'goalMouthLocation': 'low-right',
   'goalMouthCoordinates': {'x': 0, 'y': 57.35798477683012, 'z': 1.3},
   'blockCoordinates': {'x': 0.5985928964379852,
    'y': 17.45642647525029,
    'z': 0},
   'xg': 0.1557479351759,
   'xgot': 0.46516072750092,
   'id': 3564410,
   'time': 87,
   'addedTime': 5,
   'timeSeconds': 5641,
   'draw': {'start': {'x': 0.06824687069128998, 'y': 6.1986103027721775},
    'block': {'x': 65.4424454169766

In [31]:
type(shots)

dict

In [33]:
shots.keys()

dict_keys(['shotmap'])

In [71]:
shotmap_value = shots['shotmap']
print(type(shotmap_value))

<class 'list'>


In [73]:
if isinstance(shotmap_value, list):
    print(shotmap_value[:5])  # İlk 5 elemanı gösterir.
elif isinstance(shotmap_value, dict):
    print(list(shotmap_value.items())[:5])  # İlk 5 anahtar-değer çiftini gösterir.
else:
    print(shotmap_value) 

[{'player': {'name': 'Youssef En-Nesyri', 'firstName': '', 'lastName': '', 'slug': 'youssef-en-nesyri', 'shortName': 'Y. En-Nesyri', 'position': 'F', 'jerseyNumber': '19', 'userCount': 24984, 'id': 846470, 'fieldTranslations': {'nameTranslation': {'ar': 'النصيري, يوسف'}, 'shortNameTranslation': {'ar': 'ي. النصيري'}}}, 'isHome': True, 'shotType': 'save', 'situation': 'assisted', 'playerCoordinates': {'x': 2.1219979249365046, 'y': 59.55701013424094, 'z': 0}, 'bodyPart': 'left-foot', 'goalMouthLocation': 'low-right', 'goalMouthCoordinates': {'x': 0, 'y': 59.030227404759174, 'z': 1.3}, 'blockCoordinates': {'x': 1.3472298043422906, 'y': 73.95273967916373, 'z': 0}, 'xg': 0.1557479351759, 'xgot': 0.46516072750092, 'id': 3564410, 'time': 88, 'addedTime': 5, 'timeSeconds': 5641, 'draw': {'start': {'x': 63.40673344953758, 'y': 17.895750836585496}, 'block': {'x': 41.050449734951506, 'y': 0.6282131508273692}, 'end': {'x': 48.77203152923909, 'y': 0}, 'goal': {'x': 92.74138322260784, 'y': 100}}, 're

In [75]:
import pandas as pd

# 'shotmap' listesini DataFrame'e dönüştürme
df = pd.DataFrame(shots['shotmap'])

# 'player' ve 'playerCoordinates' gibi iç içe geçmiş sütunları ayırma
player_df = pd.json_normalize(df['player'])
player_coordinates_df = pd.json_normalize(df['playerCoordinates'])
goal_mouth_coordinates_df = pd.json_normalize(df['goalMouthCoordinates'])
draw_df = pd.json_normalize(df['draw'])

# Bu DataFrame'leri ana DataFrame'e birleştirme
df = pd.concat([df, player_df, player_coordinates_df, goal_mouth_coordinates_df, draw_df], axis=1)

# Gerekli sütunları düşürme
df = df.drop(['player', 'playerCoordinates', 'goalMouthCoordinates', 'draw'], axis=1)

# Yeni sütun isimlerini düzenleme
df.columns = [col.replace('.', '_') for col in df.columns]

print(df.head())

   isHome shotType  situation    bodyPart goalMouthLocation  \
0    True     save   assisted   left-foot         low-right   
1    True     miss  set-piece  right-foot        close-high   
2    True     save   assisted   left-foot         high-left   
3   False     miss   assisted        head              high   
4   False     miss   assisted   left-foot        close-left   

                                    blockCoordinates        xg      xgot  \
0  {'x': 1.3472298043422906, 'y': 73.952739679163...  0.155748  0.465161   
1                                                NaN  0.041817  0.000000   
2  {'x': 5.003178657519918, 'y': 62.7376790611554...  0.167886  0.263054   
3                                                NaN  0.440927  0.000000   
4                                                NaN  0.040331  0.000000   

        id  time  ...          y     z    start_x    start_y    block_x  \
0  3564410    88  ...  59.030227   1.3  63.406733  17.895751  41.050450   
1  3564387    

In [77]:
df.columns

Index(['isHome', 'shotType', 'situation', 'bodyPart', 'goalMouthLocation',
       'blockCoordinates', 'xg', 'xgot', 'id', 'time', 'addedTime',
       'timeSeconds', 'reversedPeriodTime', 'reversedPeriodTimeSeconds',
       'incidentType', 'goalType', 'name', 'firstName', 'lastName', 'slug',
       'shortName', 'position', 'jerseyNumber', 'userCount', 'id',
       'fieldTranslations_nameTranslation_ar',
       'fieldTranslations_shortNameTranslation_ar', 'x', 'y', 'z', 'x', 'y',
       'z', 'start_x', 'start_y', 'block_x', 'block_y', 'end_x', 'end_y',
       'goal_x', 'goal_y'],
      dtype='object')

In [79]:
df[["name", "xg", "time","id", "shotType","userCount", "shortName"]].sort_values(by="time", ascending=False)

,name,xg,time,id,id,shotType,userCount,shortName
1,Yusuf Barası,0.041817,98,3564387,1000187,miss,365,Y. Barası
0,Youssef En-Nesyri,0.155748,88,3564410,846470,save,24984,Y. En-Nesyri
2,Youssef En-Nesyri,0.167886,85,3564356,846470,save,24984,Y. En-Nesyri
3,Youssef En-Nesyri,0.440927,79,3564339,846470,miss,24984,Y. En-Nesyri
5,Andreaw Gravillon,0.033146,79,3564281,827934,block,266,A. Gravillon
4,Nabil Alioui,0.040331,77,3564322,873049,miss,528,N. Alioui
7,Edin Džeko,0.049499,62,3564212,14990,block,20728,E. Džeko
6,Allan Saint-Maximin,0.011230,57,3564248,347010,save,9779,A. Saint-Maximin
9,Edin Džeko,0.039411,55,3564194,14990,miss,20728,E. Džeko
10,Edin Džeko,0.014838,49,3564119,14990,block,20728,E. Džeko
